# Crawling

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import math
import re
import pandas as pd

In [2]:
# Settings
SONG = [] ; ALBUM = [] ; DATE = [] ; GENRE = [] ; LIKECNT = [] ; COMMENTCNT = [] ; LYRIC = [] ; WRITER = []
MELON_URL = 'https://www.melon.com/' ; QUERY = '아이유'

In [3]:
def GetAllInfos():
    
    global SONG, ALBUM, DATE, GENRE, LIKECNT, COMMENTCNT, LYRIC, WRITER
        
    # HTML 파싱
    page = driver.page_source
    soup = BeautifulSoup(page, 'lxml')
        
    # 곡명, 앨범명, 발매일, 장르, 좋아요, 댓글, 가사, 작사/작곡 수집
    song = soup.find(attrs={'class':'song_name'}).text.replace('\n', '').replace('\t', '').replace('곡명', '')
    SONG.append(song)
    album = soup.find(attrs={'class':'meta'}).find_all('dd')[0].text
    ALBUM.append(album)
    date = soup.find(attrs={'class':'meta'}).find_all('dd')[1].text
    DATE.append(date)
    genres = soup.find(attrs={'class':'meta'}).find_all('dd')[2].text
    GENRE.append(','.join(genres.split(', ')))
    likecnt = soup.find('span', attrs={'id':'d_like_count'}).text
    LIKECNT.append(likecnt)
    commentcnt = soup.find(attrs={'id':'revCnt'}).text
    COMMENTCNT.append(commentcnt)
    lyric = soup.find(attrs={'class':'lyric on'}).text.strip()
    LYRIC.append(lyric)
    writers = soup.find(attrs={'class':'list_person clfix'}).find_all(attrs={'class':'ellipsis artist'})
    WRITER.append(','.join(list(set([w.text.strip() for w in writers]))))

In [4]:
# 드라이버 생성 및 url 호출
options = webdriver.ChromeOptions()
options.add_argument('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15')
driver = webdriver.Chrome('chromedriver')
driver.get(MELON_URL)

In [5]:
# 아이유 검색
driver.find_element_by_xpath('//*[@id="top_search"]').clear()
driver.find_element_by_xpath('//*[@id="top_search"]').send_keys(QUERY)
time.sleep(2)
driver.find_element_by_xpath('//*[@id="ui-id-2"]').click()

In [6]:
# 곡 탭 클릭
driver.find_element_by_xpath('//*[@id="conts"]/div[3]/ul/li[3]/a').click()

In [7]:
totalCnt_element = driver.find_element_by_xpath('//*[@id="sort_layer"]/a[1]/span[2]').text
totalCnt = int(re.compile('[0-9]+').search(totalCnt_element).group()) # 발매곡 수
pageNum = math.ceil(totalCnt/50) # 페이지 수

In [8]:
cnt = 0
for page in range(1, pageNum+1):
    for i in range(1, 51):
        cnt += 1
        
        try:
            time.sleep(2)
            # 곡 상세 페이지로 이동
            driver.find_element_by_xpath('//*[@id="frm"]/div/table/tbody/tr['+str(i)+']/td[3]/div/div/a[1]').click()
            # 펼치기 버튼 클릭
            driver.find_element_by_xpath('//*[@id="lyricArea"]/button').click()
            # 데이터 수집
            GetAllInfos()
            # 뒤로 가기
            driver.back()

        except Exception as e:    
            print(page, i)
            print(e)
        
        if cnt==totalCnt:
            break
            
    if page<pageNum:
        driver.find_element_by_xpath('//*[@id="pageObjNavgation"]/div/span/a['+str(page)+']').click() # 다음 페이지

In [9]:
# 드라이버 종료
driver.quit()

In [10]:
# 데이터프레임 생성
data = pd.DataFrame({'SongName':SONG, 'AlbumName':ALBUM, 'Date':DATE, 'Genre':GENRE, 'LikeCnt':LIKECNT, 'CommentCnt':COMMENTCNT, 'Lyrics':LYRIC, 'Writers':WRITER})
data

,SongName,AlbumName,Date,Genre,LikeCnt,CommentCnt,Lyrics,Writers
0,라일락,IU 5th Album 'LILAC',2021.03.25,댄스,"157,405","1,736개",나리는 꽃가루에 눈이 따끔해 (아야)눈물이 고여도 꾹 참을래내 마음 한켠 비밀스런 ...,"임수호,Dr.JO,아이유,N!ko,웅킴"
1,Flu,IU 5th Album 'LILAC',2021.03.25,댄스,"51,991",346개,"Doc, I’m feeling bad 미열이 흐르고 또 어질어질해there’s to...","Martin Coogan,Zacchariah Palmer,Ryan S. Jhun,J..."
2,Coin,IU 5th Album 'LILAC',2021.03.25,R&B/Soul,"71,621","1,035개",강자에게 더 세게 I love gamble과감할수록 신세계 on my tableI’...,"아이유,Poptime,Kako"
3,봄 안녕 봄,IU 5th Album 'LILAC',2021.03.25,발라드,"56,333",724개,아프던 너의 이름도이제는 미련이 아냐하얗게 잊어버린 듯 살다가또 한 번 마주치고 싶...,"나얼,아이유,강화성"
4,Celebrity,IU 5th Album 'LILAC',2021.03.25,댄스,"270,247","3,794개","세상의 모서리구부정하게 커버린골칫거리 outsider걸음걸이, 옷차림, 이어폰 너머...","Jeppe London Bilsby,Ryan S. Jhun,Celine Svanba..."
...,...,...,...,...,...,...,...,...
173,너랑 나,2012 MelOn Music Awards Nominees,-,발라드,"9,263",8개,시곌 보며 속삭이는 비밀들간절한 내 맘속 이야기지금 내 모습을 해쳐도 좋아나를 재촉...,"이민수,김이나"
174,사랑이 잘 (With 오혁),Palette,2017.04.21,R&B/Soul,"5,623",15개,미리 말할게 사과는 안 해아무 말 없이 너 후회 안 해다 관심 없잖아 친구야 뭐야O...,"이종훈,오혁,아이유"
175,밤편지,Palette,2017.04.21,발라드,"5,505",33개,이 밤 그날의 반딧불을 당신의 창 가까이 보낼게요음 사랑한다는 말 이에요나 우리의 ...,"김희원,아이유,제휘"
176,봄 사랑 벚꽃 말고,HIGH4 1st Mini Album 'HI HIGH',2014.08.27,발라드,"8,701",5개,길었던 겨우내 줄곧 품이 좀 남는밤색 코트그 속에 나를 쏙 감추고 걸음을재촉해 걸었...,"이종훈,이채규,아이유"


In [11]:
data.to_csv('../data/data.csv')